### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [1]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType ,SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
# from statemachine import StateMachine, State
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
import logging
os.environ["PATH"] += os.pathsep + '/Users/isaacao/homebrew/bin' 
# /Users/isaacao/homebrew/bin/ '/Users/akintomiwa/homebrew/bin'
# import logging

In [3]:
from EV.agent import EV, Cpoint
import EV.model as model
from EV.statemachine import EVSM
from EV.model import get_evs_charged, get_evs_charge_level, get_evs_active, get_evs_charging, get_evs_travel, get_evs_not_idle, get_active_cpoints, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered

State diagram

Model

Model parameters

In [4]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 4
no_cps = 2

Run Model

In [ ]:
# # Jan 18 Back up - working - 2 agent classes
# model = EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
# for i in range(ticks):
#     # print(len(model.evs))
#     model.step()

In [5]:
# Jan 18 Back up - working - 2 agent classes
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
for i in range(ticks):
    # print(len(model.evs))
    model_run.step()

EV No 1 initialized. Journey type: Urban. Vehicle State: Idle
EV info: ID: 0, destination name: friend_1 , distance goal: 45, max_battery: 70, speed: 10, energy consumption rate 0.2
EV No 2 initialized. Journey type: Urban. Vehicle State: Idle
EV info: ID: 1, destination name: friend_2 , distance goal: 135, max_battery: 41, speed: 10, energy consumption rate 0.2
EV No 3 initialized. Journey type: Urban. Vehicle State: Idle
EV info: ID: 2, destination name: work , distance goal: 50, max_battery: 62, speed: 10, energy consumption rate 0.2
EV No 4 initialized. Journey type: InterUrban. Vehicle State: Idle
EV info: ID: 3, destination name: City B , distance goal: 140, max_battery: 50, speed: 20, energy consumption rate 0.5
CP info: ID: 5, initialized. Charge rate: 7.5 kW.
CP info: ID: 6, initialized. Charge rate: 7.5 kW.
Model initialised. 4 EVs and 2 Charging Points.

Current timestep (tick): 1.
Current EV state: Travel
Vehicle 1 is travelling
Current EV state: Travel
Vehicle id: 1. This 

In [84]:
run_stats = model.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charged  EVs Activated  EVs Travelling          EVs Charge Level  \
0             0              4               4  [50.0, 52.0, 33.0, 46.0]   
1             0              4               4  [48.0, 42.0, 23.0, 36.0]   
2             0              4               4  [46.0, 32.0, 13.0, 26.0]   
3             0              4               2   [44.0, 22.0, 3.0, 16.0]   
4             0              4               1  [42.0, 12.0, 10.5, 23.5]   
5             0              4               1  [40.0, 12.0, 18.0, 31.0]   
6             0              4               1  [38.0, 12.0, 25.5, 38.5]   
7             0              4               0  [36.0, 12.0, 33.0, 46.0]   
8             0              4               1  [36.0, 12.0, 40.5, 46.0]   
9             0              4               2  [36.0, 12.0, 40.5, 36.0]   
10            0              4               2  [36.0, 12.0, 30.5, 26.0]   
11            0              4               1  [36.0, 12.0, 20.5, 16.0]   
12          

In [17]:
model.datacollector.get_model_vars_dataframe().to_csv('model_output.csv')

In [ ]:
# ticks = 24 # 1 days
# no_evs = 1
# no_cps = 1
# model = EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
# # %%timeit
# for i in range(ticks):
#     # print(len(model.evs))
#     model.step()

### Batching

In [ ]:
# a = range(1000,20000,3000)
# print(type(a))
# print(a)

In [ ]:
from mesa.batchrunner import BatchRunner

# EVcounts = (1,2,3,4)
EVcounts = (100,500,1000,5000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
print(batch_df)

### Stress testing

High volume EV counts - 10,000 and 50,000

In [ ]:
ticks = 24 # 10 days
no_evs = 10000
no_cps = 1
model = EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
# %%timeit
for i in range(ticks):
    # print(len(model.evs))
    model.step()

In [ ]:
ticks = 24 # 10 days
# no_evs = 20000
no_cps = 1
model = EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
# %%timeit
for i in range(ticks):
    # print(len(model.evs))
    model.step()

In [ ]:
# %timeit L = [n ** 2 for n in range(1000)]

In [ ]:
run_stats = model.datacollector.get_model_vars_dataframe()
print(run_stats)

In [ ]:
# # model.datacollector.get_agent_vars_dataframe()
# model.datacollector.get_model_vars_dataframe()

In [ ]:
run_stats = model.datacollector.get_model_vars_dataframe()
print(run_stats)

### Visualisations

State machine based

In [ ]:
fig, (ax1) = plt.subplots(1, 1)
fig.figure.set_figwidth(12)
fig.figure.set_figheight(8)
fig.suptitle(f'Simulations stats using {no_evs} EVs', fontsize=20)
ax1.plot(run_stats[['EVs Activated',
                'EVs Currently charging',
                'EVs Travelling'
               ]])
ax1.legend(['EVs Activated',
            'EVs Currently charging',
            'EVs Travelling',
            ])
ax1.set_ylabel('EVs')
ax1.set_xlim(0)
ax1.set_ylim(0)
fig.show()

In [ ]:
# EV values
# ev_pops = [100, 1000, 10000]
ev_pops = [50, 100]
full_stats = {}
for no_evs in ev_pops:
    model = EVModel(ticks=ticks, no_evs=no_evs, no_cps=no_cps)
    for i in range(ticks):
        model.step()   
    run_stats = model.datacollector.get_model_vars_dataframe()
    full_stats[no_evs] = run_stats.iloc[-1]
    
pd.DataFrame(full_stats).transpose().astype(int)

In [ ]:
# fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
# fig.figure.set_figwidth(12)
# fig.figure.set_figheight(16)
# fig.suptitle(f'Simulations stats using {no_evs} EVs', fontsize=20)
# ax1.plot(run_stats[['EVs Activated',
#                 'EVs Charged',
#                 'EVs Travelling'
#                ]])
# ax1.legend(['EVs Activated',
#             'EVs Charged',
#             'EVs Travelling',
#             ])
# ax1.set_ylabel('EVs')
# ax1.set_xlim(0)
# ax1.set_ylim(0)
# ax2.plot(run_stats['Average Queue Size'], color='red')
# ax2.legend(['Average Queue Size'])
# ax2.set_ylabel('Customers')
# ax2.set_xlim(0)
# ax2.set_ylim(0)
# ax3.plot(run_stats['EVs Travelling'], color='grey')
# ax3.legend(['EVs Travelling (across full day    )'])
# ax3.set_ylabel('Hours')
# ax3.set_xlim(0)
# ax3.set_ylim(0)
# # ax4.plot(run_stats[['Gross Margin',
# #                 'Operating Costs',
# #                 'Total Profit'
# #                ]])
# # ax4.legend(['Gross Margin',
# #             'Operating Costs',
# #             'Total Profit'
# #             ])
# # ax4.set_ylabel('Dollars')
# # ax4.set_xlim(0)
# fig.show()

In [ ]:
# from mesa.visualization.modules import CanvasGrid
# from mesa.visualization.ModularVisualization import ModularServer

# def agent_portrayal(agent):
#     portrayal = {"Shape": "circle",
#                  "Filled": "true",
#                  "Layer": 0,
#                  "Color": "red",
#                  "r": 0.5}
#     return portrayal

# grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
# server = ModularServer(Model,
#                        [grid],
#                        "My Model",
#                        {'n_agents': 10})
# server.launch()

### Scrapbook

In [ ]:
# self._pos_init = None #unused Fix in v.0.2

# set tick distance
# 15/01: For now, tick = 1 hour. Consider changing to 30 miles per tick. 
# Change to self._speed for dynamic tick distance.
# self.tick_distance = 10 # 10 mile per tick. ??
# speed == distance per tick
# if EV active over multiple days, use learnt value from prev day. If overnight charge, set to 100. write methods to handle cases later.
# dynamic tick distance uses _speed = tick_distance
# self._chosen_cpoint_idx = 0

In [ ]:
  # Option 1: use weights to determine destination
        # define weights used
        # destinations_weights = {'work': 40, 'market': 10, 'friend_1': 15, 'friend_2': 20, 'autoshop': 15} #probability
        # assign distances to each destination
        # destination = random.choices(list(destinations_weights.keys()), weights=list(destinations_weights.values()), k=1)

In [ ]:
    # 3 Feb 23
    
    # def charge_simple(self):
    #     """Simple charging function. Charges the EV according to the charge rate."""
    #     # flags
    #     self._is_charging = True
    #     self.battery += self._charge_rate
    #     print("Vehicle " + str(self.unique_id + 1 ) + " charged status: " + str(self._was_charged)+ ". CLevel: " + str(self.battery))
    #     # self._was_charged = True




        # ## 
        # # Transition Case 6: Start charging. In_queue -> charge
        # if self.machine.state == 'In_queue':
        #     if self.battery < self._soc_charging_thresh:
        #         self.machine.start_charge()
        #         # self.charge_simple()
        #         self._chosen_cp.charge_ev()
        #         # instead, join queue, select cp, then start charge
        #     else:
        #         self.machine.end_charge()


# Old charge method from Cpoint
# def charge_ev(self):
    #     """Charge the EV at the CP.
    #     The EV is charged at the CP's charge rate.
    #     """
    #     # Transition Case 6: EV is charging at CP.
    #     if self.active_ev is not None:
    #         self.active_ev.battery += self._charge_rate
    #         # self.active_ev.machine.state == 'Charge'
    #         self.active_ev.machine.start_charge()
    #         print(f"Current EV {str(self.active_ev.unique_id+1)} at CP {self.unique_id} is in state: {self.active_ev.machine.state} ")
    #         # print("Vehicle " + str(self.active_ev.unique_id + 1 ) + " is charging status: " + str(self._was_charged)+ ". CLevel: " + str(self.active_ev.battery))


In [ ]:
# probability distribution exploration Dec 22

# Method 1
# # distr = ss.norm.rvs(size=1000,loc=12,scale=2, random_state = 42)
# # print(distr[34])
# x = np.arange(1, 25)
# xU, xL = x + 0.5, x - 0.5 
# prob = ss.norm.cdf(xU, scale = 3) - ss.norm.cdf(xL, scale = 3)
# prob = prob / prob.sum() # normalize the probabilities so their sum is 1
# nums = np.random.choice(x, size = 10000, p = prob)
# plt.hist(nums, bins = len(x))

# Method 2
# import numpy as np
# from scipy.stats import truncnorm
# import matplotlib.pyplot as plt

# scale = 3.
# range = 10
# size = 100000

# X = truncnorm(a=-range/scale, b=+range/scale, scale=scale).rvs(size=size)
# X = X.round().astype(int)
# bins = 2 * range + 1
# plt.hist(X, bins)

In [ ]:
# generate random numbers from N(0,1) # normal distribution for start time - 5am to 8pm
# type -> array of 1000 elements
# data_normal = ss.norm.rvs(size=1000,loc=12,scale=2, random_state = 42)
# print(data_normal[50])

In [ ]:
# initial approach to EV agent step method
    # approach works, little quirks. control flow faulty. does not allow for stepwise charging
    
    # if self.model._current_tick >= self.start_time:
    # while (self.odometer < self._distance_goal) & self.soc > self._soc_thresh: # didnt work. 
    # self.select_journey_type()
    # if self.odometer < self._distance_goal:
    #     self.travel()
    #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle travelled: " + str(self.odometer) + " distance units")
    #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
    #     if self.odometer == self._distance_goal:
    #         self._journey_complete == True
    #         print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
    #         self._is_active == False
    #         self._is_travelling == False
    #         self._in_garage == True

    # if (self.soc < self._soc_thresh) & (self._is_travelling == True):
    #     self._is_travelling == False
    #     print("Vehicle "+ str(self.unique_id + 1)  + " has hit SOC threshold. Heading to charging station")
    #     self._is_charging == True
    #     self.charge() #charge
    #     self.travel()
            

In [ ]:
# from travel

        # print("Vehicle id: " + str(self.unique_id) + ". This vehicle travelled: " + str(self.odometer) + " distance units")
        # print("Vehicle id: " + str(self.unique_id) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
            
        # if self.soc < self._soc_thresh:
        #     self._is_travelling == False
        #     self.charge() #charge



In [ ]:
    
    # From EV class
    # def charge_main(self):                                                                                            # For V 0.2
    #     # self._chosen_cp.active_car = None
    #     self.check_for_cp()
    #     if CP.free_cp:
    #         self.takeCP()
    #     else:
    #         # wait_time = 0
    #         self.wait_time += 1
    #         self.waitInQueue()
    #         # self._in_queue = True
    #         # self._chosen_cp.queue.append(self)

        
    #     print("Vehicle " + str(self.unique_id + 1 ) + " charging status: " + str(self._was_charged)+ ". CLevel: " + str(self.soc))

    # def charge_overnight(self):                                                                                        # For V 0.3
    #     if self._in_garage == True & self.soc < self._max_battery:
    #         # self.soc = self._max_battery
    #         self.soc += self._charge_rate
    #     self._is_charging = True
    #     self._was_charged = True

        # unused function
    # def check_for_cp(self):
    #     """Checks for a free CP in the vicinity. If found, calls select_cp."""
    #     # self.define_vicinity() or 
    #     for agent in self.model.schedule.agents:
    #         if type(Agent) == Cpoint and (agent.free_cp == True):
    #             return agent
    # unused function
    # def takeCP(self):
    #     """Signifies that CP is taken and charging begins."""
    #     for agent in self.model.schedule.agents:
    #         if type(agent) == Cpoint:
    #             agent.free_cp = False
    #             print("Vehicle " + str(self.unique_id + 1 ) + " is charging at " + str(agent))
    #             return agent.free_cp
            

In [ ]:
# if a.state == 'Idle':
#     a.start_travel()
#     print(a.state)
# print(a.state)
# ///
# a.state
# a.start_travel()
# a.state
# a.get_low()
# a.state
# a.join_charge_queue()
# a.state
# a.start_charge()
# a.state
# a.end_charge()
# a.state
# a.continue_travel()
# a.state
# a.end_travel()
# a.state


In [ ]:
# # ext

# import random

# from mesa import Agent, Model
# from mesa.time import RandomActivation

# class EV(Agent):
#     def __init__(self, unique_id, model):
#         super().__init__(unique_id, model)
#         self.battery = 100 # start with a fully charged battery
#         self.distance_to_travel = random.randint(50, 150) # set random distance to travel

#     def step(self):
#         # if battery is low, go to charging station
#         if self.battery < 20:
#             self.go_to_charging_station()
#         # if there is still distance to travel and the battery is not low, drive
#         elif self.distance_to_travel > 0:
#             self.drive()
#         # if there is no distance left to travel, do nothing
#         else:
#             return

#     def drive(self):
#         self.distance_to_travel -= 1
#         self.battery -= 1

#     def go_to_charging_station(self):
#         self.battery = 100 # fully charge the battery

# class EVModel(Model):
#     def __init__(self, num_evs):
#         self.num_evs = num_evs
#         self.schedule = RandomActivation(self)
#         # create agents
#         for i in range(self.num_evs):
#             ev = EV(i, self)
#             self.schedule.add(ev)


    # def select_journey_type(self):
    #     self._is_active == True
    #     if self.journey_choice == True:
    #         self._distance_goal = 50
    #         self.journey_type = "Urban"
    #     else:
    #          self._distance_goal = 100
    #          self.journey_type = "InterUrban"
    #     if self._is_travelling == True & self._is_active == True:
    #         print("This car's id is: " + str(self.unique_id) + " and it's going on an " + str(self.journey_type) + " journey" + " with a distance goal of " 
    #         + str(self._distance_goal))
    #     # print("Journey type selected: " + str(self._distance_goal))



In [ ]:
    # cpgen
    # def dequeue(self):
    #     if len(self.queue) > 0:
    #         self.active_ev = self.queue.pop(0)
    #         self.active_ev._is_charging = True
    #         self.active_ev._is_active = True
    #         self._is_charging = True
    #         self.active_ev._charge_start_time = self.model.schedule.time
    #         self.active_ev._charge_end_time = self.active_ev._charge_start_time + self.active_ev._charge_duration
    #     else:
    #         self.active_ev = None
    #         self._is_charging = False

In [ ]:
# class EvStateMachine(StateMachine):
#     """A state machine for managing status of EV agent in AB model.
#     Can be deployed as EvState object.

#     States:
#         Idle
#         Travelling
#         In Queue
#         Charging
#         Travelling Low
#     Transitions:
#         start_travel: Idle -> Travelling
#         join_charge_queue: Travelling -> In Queue
#         start_charge: In Queue -> Charging
#         end_charge: Charging -> Travelling
#         continue_travel: Travelling -> Travelling
#         end_travel: Travelling -> Idle

#         """
#     idle = State('Idle', initial=True)
#     travelling = State('Travelling')
#     in_queue = State('In Queue')
#     charging = State('Charging')
#     travelling_low = State('Travelling Low')

#     start_travel = idle.to(travelling)
#     join_charge_queue = travelling.to(in_queue)
#     start_charge = in_queue.to(charging)
#     end_charge = charging.to(travelling)
#     continue_travel = travelling.to(travelling)
#     end_travel = travelling.to(idle)
#     travel_low = travelling.to(travelling_low)
    

#     def on_start_travel(self):
#         # travel()
#         print('SM: Travelling')
#     def on_continue_travel(self):
#         # travel()
#         print('SM: Travelling continues')
#     def on_join_charge_queue(self):
#         # join_charge_queue()
#         print('SM: Joining Charge Queue')
#     def on_start_charge(self):
#         # charge()
#         print('SM: Charging')
#     def on_end_charge(self):
#         # travel()
#         print('SM: Travelling')
#     def on_end_travel(self):
#         # idle()
#         print('SM: Idle')
#     def on_travel_low(self):
#         # travel_low()
#         print('SM: Travelling Low')
    
# # class EvState(object):
# #     def __init__(self, state):
# #         self.state = state

# # make EVstate obj which defaults to idle
# # obj = EvState(state='idle')
# # ev_state = EvStateMachine(obj)
# # # ev_state.is_idle


# #     # go = red.to(green)
# #     # def on_slowdown(self):
# #     #     print('Baba, calm dan')
# #     # def on_stop(self):
# #     #     print('hollit')
# #     # def on_go(self):
# #     #     print('Movement!')



In [ ]:
# # ////
    # def step(self):
    #     # Block A - old state machine [pySM] - not working:
    #     # Case: Start travelling. idle -> travel
    #     # if (self.odometer < self._distance_goal) & (self.soc > self._soc_usage_thresh) & (self.state.current_state == 'idle'):
    #     if self.state.current_state == EvStateMachine.idle:
    #         self.travel()
    #         # EvStateMachine.start_travel()
    #         self.state.current_state = EvStateMachine.travelling
    #         # print(self.state.current_state)
    #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
    #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
    #      # Transition Case: Jouney Complete. travel -> idle
    #     if self.odometer == self._distance_goal:
    #         self._journey_complete == True
    #         self.state.current_state = EvStateMachine.idle
    #         # self.state.current_state = EvStateMachine.end_travel()
    #         print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
    #         # self._is_active == False
    #         # self._is_travelling == False
    #     # Transition Case: Still travelling. Travel -> travel_low
    #     if self.soc < self._soc_usage_thresh:
    #         self.state.current_state = EvStateMachine.travelling_low
    #         # self.charge()
    #         print(self.state.current_state)
    #     # Transition Case: Stopped travelling, Select cp. Travel_low -> select cp
    #     if self.state.current_state == EvStateMachine.travelling_low:
    #         self.select_cp()
    #         self.state.current_state = EvStateMachine.in_queue
    #         print(self.state.current_state)
    #     # Transition Case: Start charging. In_queue -> charge
    #     if self.state.current_state == EvStateMachine.in_queue:
    #         self.charge()
    #         self.state.current_state = EvStateMachine.charging

    #     # Transition Case: Maintain charging. Charge -> charge
    #     if self.state.current_state == EvStateMachine.charging:
    #         self.charge()
    #         print(self.state.current_state)
            
    #     # Case: continue travelling. travel -> travel
    #     if self.state.current_state == EvStateMachine.travelling:
    #         self.travel()
    #         print(self.state.current_state)
    #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
    #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
    #         # a-a if distance_goal not reached,
    #         if self.odometer == self._distance_goal:
    #             self._journey_complete == True
    #             print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
    #             self._is_active == False
    #             self._is_travelling == False
    #             self._in_garage == True
    #     if (self.state.current_state == EvStateMachine.travelling_low):
    #         self.state.start_charge()
    #         print(self.state.current_state)

In [ ]:
# class Car(Agent):
#     """
#     Attr:

#     Flags:
    
    
#     """
#     def __init__(self, unique_id, model):
#         super().__init__(unique_id, model)
#         self._max_battery = 100
#         self._soc_thresh = 10
#         self._charge_rate = 20 #Vehicle soc rises at 20 per tick
#         self._in_queue = False
#         self._in_garage = False
#         self._is_charging = False
#         self._was_charged = False
#         self._is_travelling = False
#         self._journey_complete = False

#         # self._chargerate = None
#         # self._loc_init = 
#         # self._soc_init = 50 #initially 50 (+/- 10?). if over multiple days, use learnt value from prev day. If overnight charge, set to 100. write methods to handle cases later.
#         # self.queue_time = self.chargerate * self. 
#         self.soc = ss.poisson(45).rvs()
#         self._pos_init = None #Urgent Fix soon
#         self._is_active = False
#         self.odometer = 0
#         self._distance_goal = 0
#         self.journey_type = None
#         # External 
#         self.journey_choice = choice([True, False])
#         self.start_time = ss.norm.rvs(size=1000,loc=12,scale=2, random_state = 42)
#         # self.select_journey_type()
#         # self._cp_entry = None
#         # self._cp_exit = None
#         # self._charge_start = None

#     def select_journey_type(self):
#         self._is_active == True
#         if self.journey_choice == True:
#             self._distance_goal = 50
#             self.journey_type = "Urban"
#         else:
#              self._distance_goal = 100
#              self.journey_type = "InterUrban"

#         # print("Journey type selected: " + str(self._distance_goal))


#     def travel(self):
#         self._is_travelling = True
#         self.odometer += 10
#         self.soc -= 5
#         # if self._is_travelling == True & self._is_active == True:
#         if self._is_travelling == True:
#             print("This car's id is: " + str(self.unique_id) + " and it's going on an " + str(self.journey_type) + " journey" + " with a distance goal of " 
#             + str(self._distance_goal))


#     # def select_cp(self):
#     #     # self._in_queue = True
#     #     # # queue at shortest cp
#     #     # self._chosen_cp_idx = np.argmin([
#     #     #     len(cpoint.queue) for cpoint in self.model.cpoints])
#     #     # self._chosen_cp = self.model.cpoint[self._chosen_cpoint_idx]
#     #     self._in_queue = True

    
#     def charge(self):
#         self._is_charging = True
#         # self._cp_exit = self.model._current_tick
#         # self._chosen_cp.active_car = None
#         # self.soc = self._max_battery
#         # while self.soc < self._max_battery:
#         #     self.soc += self._charge_rate
#         #     if self.soc == self._max_battery:
#         #         self._is_charging = False
#         if self.soc < self._max_battery:
#             self.soc += self._charge_rate
#             print("Vehicle " + str(self.unique_id) + " is charging")
#         if self.soc == self._max_battery:
#             print("Vehicle charged")
#         self._is_charging = False
#         self._was_charged = True
                

#     def charge_overnight(self):
#         if self._in_garage == True:
#             self.soc = self._max_battery
    
#     # def step(self):
#     #     if (self._in_queue == False) & (self.model._current_tick >= self.entry_time):
#     #         self.select_cp()
#     #     elif isinstance(self.self._cp_entry, int):
#     #         if self.model._current_tick - self._cp_entry == self.charge_time:
#     #             self.travel()
#     def step(self):
#         # while (self.odometer < self._distance_goal) & self.soc > self._soc_thresh: # didnt work. 
#         if (self.odometer < self._distance_goal):
#             self.travel()
#             print("Vehicle id: " + str(self.unique_id) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
#             print("Vehicle id: " + str(self.unique_id) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
#             if self.odometer == self._distance_goal:
#                 self._journey_complete == True
#                 print("Vehicle " + str(self.unique_id) + " has completed its journey")
#                 # self._is_active == False
#                 # self._is_travelling == False

#         if self.soc < self._soc_thresh & (self._journey_complete == False):
#             self._is_travelling == False
#             print("Vehicle has hit SOC threshold. Heading to charging station")
#             # self._is_charging == True
#             self.charge() #charge
#             self.travel()
            




# class MainModel(Model):
#     """Simulation Model with cars and charging points as two types of agents, interacting."""

#     def __init__(self, no_cars, ticks):
#         # super().__init__()
#         # init with input args
#         self.ticks = ticks
#         self._current_tick = 1
#         self.no_cars = no_cars
#         # other key model attr 
#         self.schedule = RandomActivation(self)
#         # Populate model with agents
#         self.cars = []
#         for i in range(self.no_cars):
#             car = Car(i,self)
#             self.schedule.add(car)

#         self.datacollector = DataCollector(
#             model_reporters={'Cars Charged': get_cars_charged,
#                              'Cars Activated': get_cars_active})
    
#     def step(self):
#         self.datacollector.collect(self)
#         self.schedule.step()
#         self._current_tick += 1



In [ ]:
# Old command flow for agent step function 


        # if self.machine.state == 'Charge':
        #     if self.soc >= self.soc_charging_thresh:
        #         self.machine.end_charge()


            # try:
            #     self.machine.continue_travel()
            #     self.travel()
            # except self.soc < self._soc_usage_thresh:
            #     self.machine.get_low()
            #     # self.charge()
            #     print("Current EV state: " + str(self.machine.state))
            #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
            #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")

            # Transition Case: Jouney Complete. travel -> idle
        # if self.odometer == self._distance_goal:
        #     self._journey_complete == True
        #     # self.state.current_state = EvStateMachine.idle
        #     self.state.end_travel()
        #     print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
        #     # self._is_active == False
        #     # self._is_travelling == False
        # # Transition Case: Still travelling. Travel -> travel_low
        # if self.soc < self._soc_usage_thresh:
        #     self.state.get_low()
        #     self.charge()
        #     print(self.state)
        # # Transition Case: Stopped travelling, Select cp. Travel_low -> select cp
        # if self.state == 'travel_low':
        #     self.select_cp()
        #     self.state.current_state = EvStateMachine.in_queue
        #     print(self.state.current_state)
        # Transition Case: Start charging. In_queue -> charge
        # if self.state.current_state == EvStateMachine.in_queue:
        #     self.charge()
        #     self.state.current_state = EvStateMachine.charging

        # # Transition Case: Maintain charging. Charge -> charge
        # if self.state.current_state == EvStateMachine.charging:
        #     self.charge()
        #     print(self.state.current_state)
            
        # # Case: continue travelling. travel -> travel
        # if self.state.current_state == EvStateMachine.travelling:
        #     self.travel()
        #     print(self.state.current_state)
        #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
        #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
        #     # a-a if distance_goal not reached,
        #     if self.odometer == self._distance_goal:
        #         self._journey_complete == True
        #         print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
        #         self._is_active == False
        #         self._is_travelling == False
        #         self._in_garage == True
        # if (self.state.current_state == EvStateMachine.travelling_low):
        #     self.state.start_charge()
        #     print(self.state.current_state)

        #  Block A - another approach
        # a. Once agents have been activated, check odometer and soc, then travel.
        # if (self.odometer < self._distance_goal) & (self.soc > self._soc_usage_thresh):
        #     self.travel()
        #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
        #     print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.soc) + " kwh")
        #     # a-a if distance_goal not reached,
        #     if self.odometer == self._distance_goal:
        #         self._journey_complete == True
        #         print("Vehicle " + str(self.unique_id + 1) + " has completed its journey")
        #         self._is_active == False
        #         self._is_travelling == False
        #         self._in_garage == True
        
        # Block B - state machine

        # if (self.odometer < self._distance_goal) & (self.soc <= self._soc_usage_thresh) & (self.state == 'travelling'):
        #     print("Vehicle " + str(self.unique_id + 1)  + " has hit SOC usage threshold. Heading to charging station")
        #     self.charge()
        #     self._is_charging = False
        #     print("Vehicle id: " + str(self.unique_id + 1) + " is charging. This vehicle's current charge level is: " + str(self.soc) + " kwh")
        
        # BACKUP
        # # Block B - first approach
        # if (self.odometer < self._distance_goal) & (self.soc <= self._soc_usage_thresh) & (self._is_travelling == True):
        #     print("Vehicle " + str(self.unique_id + 1)  + " has hit SOC usage threshold. Heading to charging station")
        #     # self._is_travelling == False
        #     # self._is_active = False
        #     # self.charge() #charge
        #     # if self.soc > self.soc_charging_thresh:
        #     #     self._is_charging == False
        #     #     self.travel()
        #     # else:
        #     #     self.charge()

        #     self.charge()
        #     self._is_charging = False
        #     print("Vehicle id: " + str(self.unique_id + 1) + " is charging. This vehicle's current charge level is: " + str(self.soc) + " kwh")
        #     # if (self.soc < self.soc_charging_thresh):
        #     # # if (self.soc < self.soc_charging_thresh) & (self._is_charging == True):
        #     #     self.charge()
        #     #     print("Vehicle id: " + str(self.unique_id + 1) + " is still charging. This vehicle's current charge level is: " + str(self.soc) + " kwh")
        # # else:
        # #     self.travel()
        # # if (self.odometer < self._distance_goal) & (self.soc <= self._soc_usage_thresh) & (self._is_travelling == True) & (self.soc <self.soc_charging_thresh)x
                

In [ ]:
  # energy consumption varies by speed
    # def energy_usage_tick(self):
    #     # usage = (self._speed * self.tick_distance / self._max_battery)
    #     usage = (self._speed * self.tick_distance)
    #     return usage

    # energy consumption varies by speed
    # def energy_consumption(self):
    #     consumption = (self.soc / self.usage_rate)
    #     return consumption
    # def travel(self):
    #     self.odometer += 10
    #     self.soc -= 5
    #     print("Vehicle " + str(self.unique_id + 1 ) + " is travelling")

In [ ]:
# # week of Jan 23 - 29

# class EV(Agent):
#     def __init__(self, unique_id, model):
#         super().__init__(unique_id, model)
#         self._charge_rate = 7200 # 7.5kW
#         self._in_queue = False
#         self._in_garage = False
#         self._is_charging = False
#         self._was_charged = False
#         self._is_travelling = False
#         self._journey_complete = False
#         self.machine= EVSM(initial='Idle', states=states, transitions=transitions)
#         # self._soc_init = 50 #initially 50 (+/- 10?). if over multiple days, use learnt value from prev day. If overnight charge, set to 100. write methods to handle cases later.
#         # self.queue_time = self.chargerate * self. 
#         self._pos_init = None #Urgent Fix soon
#         self._is_active = True
#         self.odometer = 0
#         self._distance_goal = None
#         self.journey_type = None
#         # External 
#         self.journey_choice = choice([True, False])
#         # For interaction with other agent classes - CPs, garages, etc.:
#         # self.start_time = ss.norm.rvs(size=1000,loc=12,scale=2, random_state = 42)
#         # self.entry_time = np.int(ss.beta(3, 3).rvs() * ticks) + 1
#         # self._cp_entry = None
#         # self._cp_exit = None
#         # self._charge_start = None
#         # Time of arrival at queue
#         # self.soc = random.randint(40, 70)
#         self.battery = random.randint(40, 70) #kWh
#         self._max_battery = self.battery
#         self._soc_usage_thresh = (0.3 * self._max_battery) # battery soc level at which EV driver is comfortable with starting charging at station.
#         self.soc_charging_thresh = (0.8 * self._max_battery) # battery soc level at which EV driver is comfortable with stopping charging at station.
#         self._chosen_cpoint = None
#         # Newest
#         self.tick_energy_usage = 0

#         # set distance_goal #Hardcoded for now
#         if self.journey_choice == True:
#             self._distance_goal = 100 #miles
#             self.journey_type = "Urban"
#         else:
#             self._distance_goal = 300 #miles
#             self.journey_type = "InterUrban"
#         # set speed
#         self._speed = 0
#         self.base_speed = 10 #urban speed (mph). Interurban speed is scaled by 2.
#         if self.journey_type == "Urban":
#             self._speed = self.base_speed
#         else:
#             self._speed = (self.base_speed * 2) #interurban speed (mph). 
#         # set vehicle energy consumption rate
#         if self.journey_type == "Urban":
#             self.ev_consumption_rate = 0.2 # 200 Wh/mile OR 20 kWh/100 miles OR 0.2 kWh/mile
#         else:
#             self.ev_consumption_rate = 0.5 # 500 Wh/mile OR 50 kWh/100 miles
#         # set tick distance
#         # 15 Jan: For now, tick = 1 hour. Consider changing to 30 miles per tick. 
#         # Change to self._speed for dynamic tick distance.
#         # self.tick_distance = 10 # 10 mile per tick. ??
#         # speed == distance per tick
    
#         # dynamic tick distance uses _speed = tick_distance
#         # self._chosen_cpoint_idx = 0
#         self._chosen_cp_idx = 0 #in selected_cp
#         self._chosen_cp = 0 #in selected_cp correct

#         #

#         print("EV No " + str(self.unique_id + 1) + " initialized. Journey type: " + str(self.journey_type) +
#         ". Vehicle State: " + str(self.machine.state) )

#         print(f"EV info: ID: {self.unique_id}, distance goal: {self._distance_goal}, max_battery: {self._max_battery}, speed: {self._speed}, energy consumption rate {self.ev_consumption_rate}")
#         # End initialisation
#     # energy consumption from distance covered
#     def energy_usage_trip(self):
#         usage = (self.ev_consumption_rate * self.odometer)
#         return usage

#     def energy_usage_tick(self):
#         usage = (self.ev_consumption_rate * self._speed)
#         # self.tick_energy_usage = usage
#         return usage

#     def delta_battery_neg(self):
#         # delta = ((self.battery - self.tick_energy_usage)/self._max_battery)
#         delta = (self.tick_energy_usage / self._max_battery)
#         return delta
  
#     def travel(self):
#         self.odometer += self._speed # old: self.tick_distance
#         # self._distance_goal - self.tick_distance
#         self.battery -= self.energy_usage_tick()
#         print("Vehicle " + str(self.unique_id + 1 ) + " is travelling")

#     def select_cp(self):
#         self._in_queue = True
#         # queue at shortest cp
#         self._chosen_cp_idx = np.argmin([len(cpoint.queue) for cpoint in self.model.cpoints])
#         print("No of active Cpoints: " + str(self.model.cpoints.count(self)))
#         self._chosen_cp = self.model.cpoints[self._chosen_cp_idx]
#         print("Vehicle " + str(self.unique_id + 1 ) + " selected: " + str(self._chosen_cp) + " for charging.")
#         self._in_queue = True
#         self._chosen_cp._is_active = True
    
    
#     # def charge_simple(self):
#     #     # alt - with sm
#     #     # self.state.start_charge()
#     #     # self.battery += self._speed
#     #     # self.battery += self._charge_rate
#     #     # 24 Jan New approach
#     #     self._chosen_cp.charge(self)
#     #     self._is_charging = True
#     #     self._was_charged = True
#     #     print("Vehicle " + str(self.unique_id + 1 ) + " charging status: " + str(self._was_charged)+ ". CLevel: " + str(self.battery))
                
#     # def charge_main(self):
#     #     self._is_charging = True
#     #     self._cp_exit = self.model._current_tick
#     #     self._chosen_cp.active_car = None
#     #     self.soc += self._charge_rate
#     #     # self._is_charging = False
#     #     self._was_charged = True
#     #     print("Vehicle " + str(self.unique_id + 1 ) + " charging status: " + str(self._was_charged)+ ". CLevel: " + str(self.soc))

    
#     # def charge_overnight(self):
#     #     if self._in_garage == True & self.soc < self._max_battery:
#     #         # self.soc = self._max_battery
#     #         self.soc += self._charge_rate
#     #     self._is_charging = True
#     #     self._was_charged = True
            
#       # ////
#     # def step(self):
#     #     # Block A - Transitions state machine:
#     #     # Transition Case 1: Start travelling. idle -> travel
#     #     if self.machine.state == 'Idle' and self.odometer < self._distance_goal:
#     #         self.machine.start_travel()
#     #         print("Current EV state: " + str(self.machine.state))
#     #     if self.machine.state == 'Travel':
#     #         self.machine.continue_travel()
#     #         self.travel()
#     #         print("Current EV state: " + str(self.machine.state))
#     #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
#     #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.battery) + " kWh")
#     #     # Transition Case 2: Still travelling, battery low. Travel -> travel_low
#     #     if self.machine.state == 'Travel' and self.battery <= self._soc_usage_thresh:
#     #         self.machine.get_low()
#     #         print("Current EV state: " + str(self.machine.state))
#     #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
#     #         print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.battery) + " wh")
#     #     # Transition Case 3: Stopped travelling, Join queue, select cp. Travel_low -> in_queue
#     #     if self.machine.state == 'Travel_low' and self.odometer < self._distance_goal:
#     #         self.machine.join_charge_queue()
#     #         self.select_cp()
#     #         print("Current EV state: " + str(self.machine.state))
#     #         print("Vehicle " + str(self.unique_id + 1 ) + " is in charging queue " + str(self._chosen_cpoint) + " . CLevel: " + str(self.battery))
#     #     # Transition Case 4: Start charging. In_queue -> charge
#     #     if self.machine.state == 'In_queue':
#     #         if self.battery < self.soc_charging_thresh:
#     #             self.machine.start_charge()
#     #             self.charge_simple()
#     #             # instead, join queue, select cp, then start charge
#     #         # else:
#     #             # self.soc >= self.soc_charging_thresh:
#     #             # self.machine.end_charge()
#     #     # Transition Case 5: Continue charging. Charge -> charge
#     #     if self.machine.state == 'Charge':
#     #         if self.battery >= self.soc_charging_thresh:
#     #             self.machine.end_charge()
#     #         else:
#     #             self.machine.continue_charge()
#     #             self.charge_simple()
#     #     # Transition Case 6: Journey Complete. travel -> idle
#     #     if self.machine.state == 'Travel' and self.odometer >= self._distance_goal:
#     #         self.machine.end_travel()
#     #         print("Current EV state: " + str(self.machine.state))
#     #         print("Vehicle " + str(self.unique_id + 1) + " has completed its journey and is now " + str(self.machine.state))
#     #     # Transition Case 7: Jouney Complete. travel_low -> idle. JIT

#     def step(self):
#         # Block A - Transitions state machine:
#         # Transition Case 1: Start travelling. idle -> travel
#         if self.machine.state == 'Idle' and self.odometer < self._distance_goal:
#             self.machine.start_travel()
#             print("Current EV state: " + str(self.machine.state))
#         if self.machine.state == 'Travel':
#             self.machine.continue_travel()
#             self.travel()
#             print("Current EV state: " + str(self.machine.state))
#             print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
#             print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.battery) + " kWh")
#         # Transition Case 2: Still travelling, battery low. Travel -> travel_low
#         if self.machine.state == 'Travel' and self.battery <= self._soc_usage_thresh:
#             self.machine.get_low()
#             print("Current EV state: " + str(self.machine.state))
#             print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle has travelled: " + str(self.odometer) + " distance units")
#             print("Vehicle id: " + str(self.unique_id + 1) + ". This vehicle's current charge level is: " + str(self.battery) + " wh")
#         # Transition Case 3: Stopped travelling, Join queue, select cp. Travel_low -> in_queue
#         if self.machine.state == 'Travel_low' and self.odometer < self._distance_goal:
#             self.machine.join_charge_queue()
#             self.select_cp()
#             print("Current EV state: " + str(self.machine.state))
#             print("Vehicle " + str(self.unique_id) + " is at CP with id: " + str(self._chosen_cpoint.unique_id) + " . CLevel: " + str(self.battery))
#         # Transition Case 4 -  from In queue to charge using existence of a chosen cp
#         # not- working : if self.machine.state == 'In_queue' and self._chosen_cpoint is not None:
#         if self.machine.state == 'In_queue':
#             if self._chosen_cp is not None:
#                 self.machine.start_charge()
#                 self._chosen_cp.charge_ev()
#                 print("Current EV state: " + str(self.machine.state))
    
#                 # print("Vehicle " + str(self.unique_id) + " is at CP with id: " + str(self._chosen_cpoint.unique_id + " . CLevel: " + str(self.battery)))
#             # self.machine.start_charge()
#             # print("Current EV state: " + str(self.machine.state))
#             # print("Vehicle " + str(self.unique_id) + " is at CP with id: " + str(self._chosen_cpoint.unique_id + " . CLevel: " + str(self.battery)))
#         # Transition Case 4: Start charging. In_queue -> charge
#         # if self.machine.state == 'In_queue':
#         #     if self.battery < self.soc_charging_thresh:
#         #         self.machine.start_charge()
#         #         self.charge_simple()
#         #         # instead, join queue, select cp, then start charge
#         #     # else:
#         #         # self.soc >= self.soc_charging_thresh:
#         #         # self.machine.end_charge()
#         # # Transition Case 5: Continue charging. Charge -> charge
#         # if self.machine.state == 'Charge':
#         #     if self.battery >= self.soc_charging_thresh:
#         #         self.machine.end_charge()
#         #     else:
#         #         self.machine.continue_charge()
#         #         self.charge_simple()
#         # # Transition Case 6: Journey Complete. travel -> idle
#         if self.machine.state == 'Travel' and self.odometer >= self._distance_goal:
#             self.machine.end_travel()
#             print("Current EV state: " + str(self.machine.state))
#             print("Vehicle " + str(self.unique_id + 1) + " has completed its journey and is now " + str(self.machine.state))
#         # Transition Case 7: Jouney Complete. travel_low -> idle. JIT
    